## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [0]:
%tensorflow_version 2.x

In [149]:
import tensorflow as tf
print(tf.__version__)

2.0.0


In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


### Collect Data

In [150]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#dataframe = read_csv("/content/drive/My Drive/AIMLDataContent/housing.csv", delim_whitespace=True, header=None)

In [0]:
import pandas as pd

In [0]:
iris = pd.read_csv("/content/drive/My Drive/AIMLDataContent/11_Iris.csv")
prices = pd.read_csv("/content/drive/My Drive/AIMLDataContent/prices.csv")

### Check all columns in the dataset

In [154]:
print(iris.columns)

Index([u'Id', u'SepalLengthCm', u'SepalWidthCm', u'PetalLengthCm',
       u'PetalWidthCm', u'Species'],
      dtype='object')


In [155]:
print(prices.columns)

Index([u'date', u'symbol', u'open', u'close', u'low', u'high', u'volume'], dtype='object')


### Drop columns `date` and  `symbol`

In [0]:
prices.drop(['date','symbol'],inplace = True,axis=1)

In [157]:
prices.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [158]:
prices['volume']=prices['volume']/1000000
prices.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2.1636
1,125.239998,119.980003,119.940002,125.540001,2.3864
2,116.379997,114.949997,114.930000,119.739998,2.4895
3,115.480003,116.620003,113.500000,117.440002,2.0063
4,117.010002,114.970001,114.089996,117.330002,1.4086


In [159]:
prices_sample = prices.head(1000)
prices_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
open      1000 non-null float64
close     1000 non-null float64
low       1000 non-null float64
high      1000 non-null float64
volume    1000 non-null float64
dtypes: float64(5)
memory usage: 39.1 KB


In [160]:
X = prices_sample.drop(['volume'],axis=1)
y = prices_sample['volume']
print(X.shape)
print(y.shape)

(1000, 4)
(1000,)


### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3,random_state=5)

#### Convert Training and Test Data to numpy float32 arrays


In [0]:
X_train.shape
import numpy as np

In [0]:
X_train_tf =np.array(X_train).astype('float32')
X_test_tf = np.array(X_test).astype('float32')
y_train =np.array(y_train).astype('float32')
y_test = np.array(y_test).astype('float32')

In [164]:
X_train_tf.dtype

dtype('float32')

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
x_n = tf.nn.l2_normalize(X_train_tf,1)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
W = tf.zeros(shape=(4, 1))
b = tf.zeros(shape=(1))

2.Define a function to calculate prediction

In [0]:
@tf.function
def prediction(x_n, W, b):
    y_pred = tf.add(tf.matmul(x_n, W), b)
    return y_pred

3.Loss (Cost) Function [Mean square error]

In [0]:
@tf.function
def loss(y_actual, y_predicted):
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    return avg

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
@tf.function
def train(x, y_actual, w, b, learning_rate=0.01):
    
    # Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        t.watch([w,b])
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    # Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    # Update Weights and Bias
    w = w - learning_rate * dw
    b = b - learning_rate * db
    
    return w, b
    

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [172]:
for i in range(100):    
    W, b = train(x_n, y_train, W, b)
    if ((i % 5)==0):
      print('Current Training Loss on iteration', i, loss(y_train, prediction(x_n, w, b)).numpy())

('Current Training Loss on iteration', 0, 237.99004)
('Current Training Loss on iteration', 5, 237.9503)
('Current Training Loss on iteration', 10, 237.91795)
('Current Training Loss on iteration', 15, 237.89153)
('Current Training Loss on iteration', 20, 237.87009)
('Current Training Loss on iteration', 25, 237.8526)
('Current Training Loss on iteration', 30, 237.83841)
('Current Training Loss on iteration', 35, 237.8267)
('Current Training Loss on iteration', 40, 237.81714)
('Current Training Loss on iteration', 45, 237.80957)
('Current Training Loss on iteration', 50, 237.80324)
('Current Training Loss on iteration', 55, 237.79822)
('Current Training Loss on iteration', 60, 237.79422)
('Current Training Loss on iteration', 65, 237.79079)
('Current Training Loss on iteration', 70, 237.78792)
('Current Training Loss on iteration', 75, 237.78558)
('Current Training Loss on iteration', 80, 237.78381)
('Current Training Loss on iteration', 85, 237.78233)
('Current Training Loss on iterat

### Get the shapes and values of W and b

In [173]:
W.numpy()

array([[1.2788088],
       [1.2834452],
       [1.265719 ],
       [1.29457  ]], dtype=float32)

In [174]:
b.numpy()

array([2.561419], dtype=float32)

### Model Prediction on 1st Examples in Test Dataset

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [175]:
iris = pd.read_csv("/content/drive/My Drive/AIMLDataContent/11_Iris.csv")
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [0]:
iris.drop(['Id'],axis=1,inplace=True)

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [177]:
iris = pd.get_dummies(iris, columns=['Species'])
iris.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,5.1,3.5,1.4,0.2,1,0,0
1,4.9,3.0,1.4,0.2,1,0,0
2,4.7,3.2,1.3,0.2,1,0,0
3,4.6,3.1,1.5,0.2,1,0,0
4,5.0,3.6,1.4,0.2,1,0,0


### Splitting the data into feature set and target set

In [178]:
X = iris.iloc[:,0:4]
y = iris.iloc[:,4:7]
print(X.shape)
print(y.shape)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3,random_state=5)

(150, 4)
(150, 3)


###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [0]:
model = tf.keras.models.Sequential()

input1 = tf.keras.layers.Input(shape=(4,))
model.add(input1)
# Normalize the data
model.add(tf.keras.layers.BatchNormalization())

# Add Dense Layer which provides 3 Outputs after applying softmax

model.add(tf.keras.layers.Dense(3, activation='softmax'))

model.compile(optimizer='SGD',
              loss='categorical_crossentropy',
              metrics=['accuracy'])





### Model Training 

In [180]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50,
          batch_size = X_train.shape[0])

Train on 105 samples, validate on 45 samples
Epoch 1/50
105/105 [==============================] - 0s 1ms/sample - loss: 0.6759 - accuracy: 0.6857 - val_loss: 1.3210 - val_accuracy: 0.3111
Epoch 2/50
105/105 [==============================] - 0s 66us/sample - loss: 0.6737 - accuracy: 0.6857 - val_loss: 1.3138 - val_accuracy: 0.3111
Epoch 3/50
105/105 [==============================] - 0s 39us/sample - loss: 0.6714 - accuracy: 0.6857 - val_loss: 1.3067 - val_accuracy: 0.3111
Epoch 4/50
105/105 [==============================] - 0s 54us/sample - loss: 0.6692 - accuracy: 0.6857 - val_loss: 1.2996 - val_accuracy: 0.3111
Epoch 5/50
105/105 [==============================] - 0s 52us/sample - loss: 0.6670 - accuracy: 0.6857 - val_loss: 1.2926 - val_accuracy: 0.3111
Epoch 6/50
105/105 [==============================] - 0s 44us/sample - loss: 0.6649 - accuracy: 0.6857 - val_loss: 1.2857 - val_accuracy: 0.3111
Epoch 7/50
105/105 [==============================] - 0s 81us/sample - loss: 0.6627 - 

### Model Prediction

In [181]:
model.predict(X_test)

array([[0.03842137, 0.19086288, 0.7707157 ],
       [0.01149932, 0.14387554, 0.8446251 ],
       [0.01539558, 0.13440225, 0.85020214],
       [0.43425182, 0.24198319, 0.323765  ],
       [0.00626645, 0.08699165, 0.90674186],
       [0.07788574, 0.24503566, 0.6770786 ],
       [0.30974486, 0.26184508, 0.42841005],
       [0.02852496, 0.15907918, 0.8123959 ],
       [0.27611884, 0.26489753, 0.4589836 ],
       [0.04382748, 0.19233386, 0.7638387 ],
       [0.03819193, 0.1836258 , 0.77818227],
       [0.01463224, 0.14312337, 0.84224445],
       [0.00517806, 0.09386299, 0.90095896],
       [0.00371334, 0.08985613, 0.9064305 ],
       [0.3334357 , 0.244025  , 0.42253926],
       [0.2850185 , 0.24794747, 0.467034  ],
       [0.0134096 , 0.13534276, 0.85124767],
       [0.00338037, 0.08612681, 0.91049284],
       [0.38616925, 0.22751407, 0.38631666],
       [0.33026028, 0.26335597, 0.4063837 ],
       [0.0312024 , 0.17407934, 0.79471827],
       [0.0085586 , 0.11576544, 0.875676  ],
       [0.

### Save the Model

In [182]:
model.save('R6_Internal_Lab_UpdatedTF2_Prices_Iris.ipynb')

INFO:tensorflow:Assets written to: R6_Internal_Lab_UpdatedTF2_Prices_Iris.ipynb/assets


### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?